In [1]:
import cv2
import numpy as np
import pathlib
import pandas as pd
import glob

In [2]:
# define the path to the target block
block_path = pathlib.Path(r'Z:\Nimrod\experiments\PV_24\27_12_2021\block_12')

# extract paths to objects of interest
le_video = [vid for vid in glob.glob(str(block_path) + r'\eye_videos\LE\**\*.mp4') if
                  "DLC" not in vid]
re_video = [vid for vid in glob.glob(str(block_path) + r'\eye_videos\RE\**\*.mp4') if
                  "DLC" not in vid]
le_analysis_path = block_path / 'analysis' / 'le_df.csv'
re_analysis_path = block_path / 'analysis' / 're_df.csv'

In [3]:
# This cell reads the analysis file and gets it ready for ellipse fitting in the video
r_df = pd.read_csv(re_analysis_path)
r_width = np.floor(r_df.width.values)

r_height = np.floor(r_df.height.values)

MmAxes_r = []
angle_r = np.floor(np.degrees(r_df.phi.values))

center_coords_r = [np.floor(r_df.center_x.values), np.floor(r_df.center_y.values)]

for i in range(len(r_width)):
    if r_width[i] == r_width[i]:
        if r_width[i] > r_height[i]:
            MmAxes_r.append((int(r_width[i]),int(r_height[i])))
        else:
            MmAxes_r.append((int(r_height[i]), int(r_width[i])))
    else:
        MmAxes_r.append((np.nan, np.nan))

In [4]:
### working ###
# This cell runs a video of the right eye with overlaid ellipses

re_cap = cv2.VideoCapture(re_video[0])
frame_n = -1
# go over frames of the video
while re_cap.isOpened():
    bad_flag = 0
    # get a frame
    re_ret, re_frame = re_cap.read()
    if re_ret is True:
        frame_n += 1
        # find the row in the dataframe that corresponds with the frame_n
        try:
            i = np.where(r_df.R_eye_frame.values.astype(np.int32) == frame_n)[0][0]
        except IndexError:
            i = 0

        re_frame = cv2.cvtColor(re_frame, cv2.COLOR_BGR2GRAY)
        if MmAxes_r[i][0] == MmAxes_r[i][0]:
            re_frame_with_ellipse = cv2.ellipse(re_frame,
                                                (int(center_coords_r[0][i]),int(center_coords_r[1][i])),
                                                (MmAxes_r[i][0], MmAxes_r[i][1]),
                                                angle_r[i],
                                                0,
                                                360,
                                                (0, 255, 0),
                                                2)
            cv2.imshow('Right eye frame', re_frame_with_ellipse)
        else:
            cv2.imshow('Right eye frame', re_frame)
        if cv2.waitKey(5) & 0xFF==ord('q'):
            break
    else:
        break
re_cap.release()

cv2.destroyAllWindows()

In [46]:
cv2.destroyAllWindows()